# Individual Assignment 3 - Querying data
In this assignment, students are going to focus on getting data out of a database, using a few different methods

## Goal
The goal of this assignment is to understand how to get data out of a database using SQL queries directly, pandas and also directly into our objects using SQLAlchemy 

## Deliverables
1. A PDF of this notebook OR a link to the repository where the final version of the notebook is store.

## Background
Relational databases are super helpful way to store data and settings for applications.  They are efficient at managing space, easy to maintain and provide an extra layer of security to the data. Being able to load and save data to a database from a program brings a great deal of flexibility to our ability to automate.

## Instructions
For this assignment, you should work through this notebook.  Executing all the cells and writing code in the various YOUR TURN cells in this notebook.

You have a choice of either submitting a PDF of the notebook output, or a link to the repository where the completed notebook is stored.

# <a id='toc2_'></a>[Part 1 - Querying data using SQL commands](#toc0_)
Since you already know quite a bit about querying a SQL database, this is the easiest way to get data from a database.

The basic steps are 
1. *Connect to the database*. This may require some setup work depending on database type (SQLite, MS SQL, Teradata, etc).  Often times an intermediate piece of software called a database driver is required. This piece of software knows how to make a connection to the database server and send your SQL commands to the database and return the results to you. For SQLite, the driver is built-into Python and so there is no extra software to setup other than the standard Python libraries.
2. *Create a cursor* that can be used to send commands to the server. A database cursor is a pointer that moves around the database and points to the data you want to retrieve from the database.
3. *Execute a command*. Once the setup has been completed, we can finally execute a command. A command is the SQL statement you want to be sent to the database.  If the command returns data (as in a SELECT statement), then the cursor will point to the data that is to be returned (but keep in mind, only 1 row of data at a time).

The next cell shows the basics of creating a connection, establishing a cursor and executing a command to the database.

In [ ]:
import sqlite3

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

# Next make a cursor that can be used to run a query on the new connection
cur = cnn.cursor()

# See what tables are available in the database
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(tables.fetchall())


# get the first 10 records from the sales table
result = cur.execute('SELECT * from sales LIMIT 10')

# Get the names of the columns from a sqlite database
names = list(map(lambda x: x[0], cur.description))
print(names)

# Print the first 10 records
for row in result:
    print(row)  

# It's good practice to ensure that we close our connection to the database when we are done using it
cnn.close()


Notice that each row of the query returns the result in a tuple.  A tuple similar to a list - it's a collection of values.  If we want to access a particular item in the results - we have to do so with an index.  Let's start by getting some data then we'll 

In [ ]:
# We have to reopen the database and create a new cursor to run queries on it
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()

# We can also use the fetchall() method to get all the results at once
results = cur.execute('SELECT * from sales LIMIT 10').fetchall()
print(results)
    
# We can also use the fetchmany() method to get a specific number of results
results = cur.execute('SELECT * from sales LIMIT 10').fetchmany(5)
print(results)

In [ ]:
# Now lets get just a single result and print out the SalesID, and the Sale Date.
results = cur.execute('SELECT * from sales LIMIT 10').fetchone()
print(results)
# Print the SalesID and Sale Date.  The SalesID is the first column, and the Sale Date is the 17th column
print(f'SalesID: {results[0]}, Sale Date: {results[16]}')

## <a id='toc2_1_'></a>[YOUR TURN - Queries](#toc0_)
Now that we know which columns are available to us we can query the database let's run a few queries.
Answer the questions in the following cells.  Ensure that your code and the result are available before saving to PDF.

As a reference, here are the column names:
```
sale_id
Configuration
Customer Postcode
Store Postcode
Retail Price
Screen Size (Inches)
Battery Life (Hours)
RAM (GB)
Processor Speeds (GHz)
Integrated Wireless?
HD Size (GB)
Bundled Applications?
customer X
customer Y
store X
store Y
sale_date
```

In [ ]:
# The following steps are provided as a reminder of how to execute the query

# Open the database connection
# Create the cursor for which to execute the query
# Execute the query
# Print the results
#  HINT: Since we are only looking for a single value - fetchone() is a good method to use
# Close the connection

# Q1: What is the total revenue from sales in the sales table? (the answer should be 144373314.0)

In [ ]:
# Q2: Sum the sales by Store Postcode include the postcode in the results
# HINT: Now we are looking for multiple values - fetchall() is a good method to use

### <a id='toc2_1_1_'></a>[Using the data from the queries](#toc0_)
In all the examples the data comes back from the query as a tuple.  This means that we are getting all the values in a sort of collection of variables.  We can access the data in tuple just like it were a list.

Now, create a class that has a property for each of the columns in the database.
Query the database to get the first 20 items in the database, create an object from each one and put it in a list
Then loop through the list and print the Sale Id, Configuration and Sale date for all 20 items in list


In [ ]:
# Define the laptop sale class
# This class will be used to store the results of the query

# # Open the database connection
# Create the cursor for which to execute the query
# Execute the query

# Create an empty list to store the results
# Loop through the results and create a new LaptopSale object for each row
# Add the new LaptopSale object to the list

# Close the connection

# Loop through the list of objects
# Print the object's Sale Id, Configuration and Sale date

# <a id='toc3_'></a>[Part 2 - Using pandas to connect to the database](#toc0_)
If we are just looking to grab some data from a database and use it for analytics especially, then the [pandas](https://pandas.pydata.org/) library is a great choice.

To get data from SQL using `pandas` all we need to do is create the connection and execute the query


In [ ]:
# Create connection
import sqlite3
import pandas as pd

# Set the name of the database file to be used for this exercise
db_file = 'laptopsales.db'

# Make a connection to the database
cnn = sqlite3.connect(db_file)

data = pd.read_sql_query("SELECT * FROM sales LIMIT 10", cnn)
data

Now that our data is in a dataframe we can do all the things we know how to do with pandas already - like aggregate, filter, group and slice

In [ ]:
# Calculate the average battery life
mean_battery_life = data['Battery Life (Hours)'].mean()
print(f'Average Battery Life: {mean_battery_life} hours')

# Get all sales where the laptop has integrated wireless
wireless_sales = data[data['Integrated Wireless?'] == 'Yes']
print(f'wireless sales count: {len(wireless_sales)}')

# Price difference between items with and without integrated wireless
wireless = data[data['Integrated Wireless?'] == 'Yes']['Retail Price'].mean()
no_wireless = data[data['Integrated Wireless?'] == 'No']['Retail Price'].mean()
print(f'Price difference between items with and without integrated wireless: {wireless - no_wireless}')


## <a id='toc3_1_'></a>[YOUR TURN - Using pandas to query data source](#toc0_)
Now that you have seen how to connect to a database using pandas, we are going to move to a new database with a few more tables.  You can read about the chinook example [here](https://www.sqlitetutorial.net/sqlite-sample-database/).

![chinook database](images/sqlite-sample-database-color.jpg)

Complete the following queries and show your results

In [ ]:
# Create connection
# Write a query that will return the artist name, album title, and genre name into a dataframe
# Close the connection
# Show the dataframe

### <a id='toc3_1_1_'></a>[Dynamic Query](#toc0_)
For this next exercise you need to ask the user for a band name and then use this name to get all the albums for that band into a dataframe.

***HINTS:***
1. Consider building the query with string manipulation
2. It might be difficult to get the band name spelled exactly right, so use the SQL operator `LIKE` to help with getting close
3. To get user input you will use the `input()` function
4. Review [this lesson](https://www.py4analytics.info/book/lessons/250-Database_Connections.html#dynamic-queries) on dynamic queries if you need some additional help

In [ ]:
artist_name = input("What band are you interested in?  Put a partial name if you don't know the full name: ")
# Your code for the dynamic query goes here

# <a id='toc4_'></a>[Part 3 - Using an SQLAlchemy (Object Relational Mapping)](#toc0_)
So far you have learned how to work with databases by querying directly, using dynamic queries and getting the data back in the form of a dataframe.  What would be super handy in some cases is to be able to map database objects directly onto Python objects. Being able to treat database tables like other Python objects is very valuable.  Expecting each row to be represented by a particular Python object allows us to access the columns as properties of the class and handles the insert/update/delete functions for us without having to depend on fragile SQL strings.  (Fragile SQL strings means that we are putting code logic into a string which isn't checked by Python before being run, so if there is a change in the database, we have to search through strings in our code to update the logic rather than being able to depend on refactoring tools provided by our IDEs.  *Are you glad you asked what 'fragile SQL strings' means?*)  The library most often used to do ORM (object relational mapping) is a library called SQLAlchemy.

Going back to our `chinook` dataset.  It would be helpful to be able to consider customers as Python objects, if we could query for them and then use the objects to build our front-end forms or maybe just export them to a semi-strucured file format like json or xml for transport.  We'll start with an example and then break it down.

**NOTE: If you haven't installed [SQLAlchemy](https://www.sqlalchemy.org/) yet you are going to need to do so.  You can reference this site for the [installation guide](https://docs.sqlalchemy.org/en/20/intro.html#installation).  Try not to get to mired in the details of the rest of the library just yet**

In [ ]:
import sqlite3
import sqlalchemy
# install SQLAlchemy using pip install SQLAlchemy
from sqlalchemy import create_engine, text

if sqlalchemy.__version__ < '2.0.0':
    raise Exception('Please upgrade your version of SQLAlchemy to 2.0.0 or greater')

engine = create_engine('sqlite:///chinook.db')
with engine.connect() as conn:
    # Get the top 10 albums
    results = conn.execute(text("SELECT * FROM albums LIMIT 10")).fetchall()

for album in results:
    print(album)

So far we see we can use SQLAlchemy in the same way we used connections to the SQLite3 database already.  Let's take it one step further.

**NOTE: It may be helpful to review the [ORM Quick Start](https://docs.sqlalchemy.org/en/20/orm/quickstart.html) provided by SQLAlchemy in conjunction with the following**

In [ ]:
from typing import List, Optional 
from sqlalchemy import Integer, String, DateTime, Float, ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Session, relationship
from sqlalchemy import select

class Base(DeclarativeBase):
    pass

# Notice we are naming the class in the singular, even though the table name is plural
# This is because each instance of the class represents a single row in the table
class Invoice(Base):
    __tablename__ = 'invoices'
    
    id: Mapped[int]= mapped_column('InvoiceId', primary_key=True)
    invoiceDate = mapped_column(DateTime)
    billingAddress = mapped_column(String)
    billingCity= mapped_column(String)
    billingState= mapped_column(String)
    billingCountry = mapped_column(String)
    billingPostalCode = mapped_column(String)
    total= mapped_column(Float)

    # Create a list to hold the invoice details associated with this invoice
    invoice_items: Mapped[List['Invoice_Item']] = relationship(back_populates='invoice')

    # This is a special method that is called when we print an instance of the class
    def __repr__(self):
        return f'Invoice(id={self.id}, invoiceDate={self.invoiceDate}, billingAddress={self.billingAddress}, billingCity={self.billingCity}, billingState={self.billingState}, billingCountry={self.billingCountry}, billingPostalCode={self.billingPostalCode}, total={self.total})'
    

# A class to represent the invoice items table
class Invoice_Item(Base):
    __tablename__ = 'invoice_items'
    
    id: Mapped[int]= mapped_column('InvoiceLineId', primary_key=True)    
    # This provides us a link back to the invoice table
    # Notice we are using the more Python appropriate invoice_id instead of InvoiceId which is the column name in the table
    invoice_id : Mapped[int]= mapped_column('InvoiceId', ForeignKey('invoices.InvoiceId'))
    invoice : Mapped[Invoice]= relationship('Invoice', back_populates='invoice_items')
    
    track_id : Mapped[int]= mapped_column('trackId', ForeignKey('tracks.TrackId'))
    track: Mapped['Track']= relationship('Track')
    
    unit_price : Mapped[float]= mapped_column('unitPrice')
    quantity : Mapped[int]= mapped_column('quantity')

# Let's add a few more classes so we can get all the details of the orders from the database
class Track(Base):
    __tablename__ = 'tracks'
    
    id: Mapped[int]= mapped_column('TrackId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')
    
    # Define the album id column and link it to the album table
    album_id: Mapped[int]= mapped_column('AlbumId', ForeignKey('albums.AlbumId'))
    album: Mapped['Album']= relationship('Album')
    
    # Define the media type id column and link it to the media type table
    media_type_id: Mapped[int]= mapped_column('MediaTypeId', ForeignKey('media_types.MediaTypeId'))
    media_type : Mapped['MediaType']= relationship('MediaType')
    
    # Define the genre id column and link it to the genre table
    genre_id: Mapped[int]= mapped_column('GenreId', ForeignKey('genres.GenreId'))
    genre : Mapped['Genre']= relationship('Genre')
    
    composer: Mapped[str]= mapped_column('Composer')
    milliseconds: Mapped[int]= mapped_column('Milliseconds')
    bytes: Mapped[int]= mapped_column('Bytes')
    unit_price: Mapped[float]= mapped_column('UnitPrice')
    
# Class that maps to MediaType
class MediaType(Base):
    __tablename__ = 'media_types'
    
    id: Mapped[int]= mapped_column('MediaTypeId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

# Class that maps to Genre
class Genre(Base):
    __tablename__ = 'genres'
    
    id: Mapped[int]= mapped_column('GenreId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

# Class that maps to Album
class Album(Base):
    __tablename__ = 'albums'
    
    id: Mapped[int]= mapped_column('AlbumId', primary_key=True)
    title: Mapped[str]= mapped_column('Title')

    artist_id: Mapped[int]= mapped_column('ArtistId', ForeignKey('artists.ArtistId'))
    artist: Mapped['Artist']= relationship('Artist')

# Class that maps to Artist
class Artist(Base):
    __tablename__ = 'artists'
    
    id: Mapped[int]= mapped_column('ArtistId', primary_key=True)
    name: Mapped[str]= mapped_column('Name')

session = Session(engine)
result = session.query(Invoice).limit(10)
for invoice in result:
    print(invoice)

# We want to ensure that we are always closing the session after we are done using it
session.close()

What's going on here. In the cell above, we create a class called ```Invoice``` which maps directly to the ```Invoice``` table in the database.  SQLAlchemy knows how to convert *objects* of type ```Invoice``` into rows in the database.  This saves us the effort and fragility of updating custom sql statements.

What's more, we can establish the relationship between ```invoice_items``` and ```invoices```, meaning we can easy find the ```invoice``` related to an ```invoice_items``` and likewise, we have a list of ```invoice_items``` represented on an ```invoice```.

Furthermore, we can leverage Python to do many of the same things we could do in SQL using SQL like functions.  Take a look at the next cell

In [ ]:
# Create a new session
session = Session(engine)

# Get the first invoice using the id
invoice_1 = session.query(Invoice).filter(Invoice.id == 1).first()
print (f'Invoice 1: {invoice_1.invoiceDate}')

# Get the first 10 invoices, using something more like SQL
statement = select(Invoice).where(Invoice.id <= 10)
for invoice in session.scalars(statement):
    print(f'Invoice {invoice.id}: {invoice.invoiceDate}')

# Close the session
session.close()


The real value of the ORM (object relational model) mode of SQLAlchemy is when there are relationships between items in the database.  For instance, if we look closer at the `invoice` and `invoice_items` tables we see they are related.  It would be super handy to be able to load up our invoice_items when we load up a specific invoice.  

<p align="center"><img src="./images/a3-invoices.png" alt="close-up of the invoices and invoice_items tables"/>
</p>
<span style="color:red">***BEWARE: The image shows that the key to the ```invoice_items``` table is called ```InvoiceItemId``` in reality it is called ```InvoiceLineId```.***</span>

In [ ]:
# Create a new session
session = Session(engine)

# Get the first invoice using the id
invoice_1 = session.query(Invoice).filter(Invoice.id == 1).first()
print (f'Invoice 1: {invoice_1.invoiceDate}')
    
invoice_items = invoice_1.invoice_items
print(f'Invoice 1 has {len(invoice_items)} invoice items')  
for invoice_item in invoice_items:
    print(f'Invoice Item {invoice_item.id}: {invoice_item.unit_price}, {invoice_item.quantity}')

# Close the session to ensure that we are not keeping a connection open
session.close()

So just by querying the invoice we also have access to the details of the invoice. Since we are interested in all the details there is no need to create complex join statements like we would before to get the information we needed.  

Now, let's add the `Track` information to round out the picture.  We are going to skip `playlists` and `customers` because they introduce some additional complexity which isn't as easy to map, but which definitely can be done.  The next cell demonstrates the pattern we've seen earlier where we define the *id* field as a foreign key and then immediately follow this with a property which holds (one or more) of the objects represented by the row in the parent table.  For instance,

```python
    media_type_id: Mapped[int]= mapped_column('MediaTypeId', ForeignKey('media_types.MediaTypeId'))
    media_type : Mapped['MediaType']= relationship('MediaType')
```

SQLAlchemy will get the media_type_id for us, but also, it is smart enough to load an object of type `MediaType` for us.  Allowing us to get the useful part, the name of the media_type rather quickly
```python
    track.media_type.name
```
*While the order of the property definitions doesn't matter, the code is easier to read if these two lines are kept together.*

In [ ]:
# Let's look for the artist name and album for the first 10 albums in the database
session = Session(engine)
statement = select(Album).limit(10)
for album in session.scalars(statement):
    print(f'Album: {album.title}, Artist: {album.artist.name}')
session.close()

## <a id='toc4_1_'></a>[YOUR TURN - Querying data using SQLAlchemy](#toc0_)
Now we are going to use a bit of what we know to get some information from the database using SQLAlchemy.  If you haven't done so a quick review of the [ORM Quickstart](https://docs.sqlalchemy.org/en/20/orm/quickstart.html) for SQLAlchemy may be helpful.  Also, you may find the [ORM Querying Guide](https://docs.sqlalchemy.org/en/20/orm/queryguide/index.html) helpful.

- Always start by writting in English (in comments) what you want to complete prior to writing code.
- Refer to the previous cells for examples


In [ ]:
# Get the track name for the first 10 tracks in the database

# Create the session
# Query the tracks
# For each track, print the track name
# Close the session

In [ ]:
# Get the track name and genre name for the first 10 tracks in the database

# Create the session
# Query the tracks
# For each track, print the track name and genre name
# Close the session
